## Clean Data

Get household columns - all columns except individuals and squared

In [1]:
from data_cleaning import clean_non_numeric_features, clean_missing_values

def clean_data(data):
    data = clean_non_numeric_features(data)
    data = clean_missing_values(data)
    return data

/home/minnie/.pyenv/versions/3.6.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/minnie/.pyenv/versions/3.6.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
from data_cleaning import get_training_data, get_missing_features, clean_non_numeric_features

data = get_training_data()
data = clean_data(data)

Loading data from data/train.csv...
(9557, 142)

Checking for inconsistent targets...
(85,)
(9557, 142)
Cleaning inconsistent targets...
Checking inconsistent targets are gone...
(0,)
(9557, 142)



In [3]:
target_column = 'Target'
person_id = 'Id'
household_id = 'idhogar'
head_of_household = 'parentesco1'

In [4]:
from column_categories import column_descriptions, individuals_info, squared
hh_columns = [k for k in data if k not in individuals_info and k not in squared]
hh_data = data.reset_index()[hh_columns]
print(hh_columns)

['v2a1', 'hacdor', 'rooms', 'hacapo', 'v14a', 'refrig', 'v18q1', 'r4h1', 'r4h2', 'r4h3', 'r4m1', 'r4m2', 'r4m3', 'r4t1', 'r4t2', 'r4t3', 'tamhog', 'tamviv', 'hhsize', 'paredblolad', 'paredzocalo', 'paredpreb', 'pareddes', 'paredmad', 'paredzinc', 'paredfibras', 'paredother', 'pisomoscer', 'pisocemento', 'pisoother', 'pisonatur', 'pisonotiene', 'pisomadera', 'techozinc', 'techoentrepiso', 'techocane', 'techootro', 'cielorazo', 'abastaguadentro', 'abastaguafuera', 'abastaguano', 'public', 'planpri', 'noelec', 'coopele', 'sanitario1', 'sanitario2', 'sanitario3', 'sanitario5', 'sanitario6', 'energcocinar1', 'energcocinar2', 'energcocinar3', 'energcocinar4', 'elimbasu1', 'elimbasu2', 'elimbasu3', 'elimbasu4', 'elimbasu5', 'elimbasu6', 'epared1', 'epared2', 'epared3', 'etecho1', 'etecho2', 'etecho3', 'eviv1', 'eviv2', 'eviv3', 'idhogar', 'hogar_nin', 'hogar_adul', 'hogar_mayor', 'hogar_total', 'dependency', 'edjefe', 'edjefa', 'meaneduc', 'bedrooms', 'overcrowding', 'tipovivi1', 'tipovivi2',

In [5]:
from data_cleaning import target_by_household
target_household_map = target_by_household(hh_data)
hh_data = hh_data.drop(target_column, axis=1).groupby(household_id).mean()
hh_data = hh_data.join(target_household_map)

In [6]:
from data_cleaning import compress_column_data
hh_data = compress_column_data(hh_data)
get_missing_features(hh_data)

Series([], dtype: float64)

In [7]:
from data_cleaning import add_custom_features
hh_data = add_custom_features(hh_data)

In [8]:
import numpy as np
# Create correlation matrix
corr_matrix = hh_data.corr()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(abs(upper[column]) > 0.95)]

print(f'There are {len(to_drop)} correlated columns to remove.')
print(to_drop)

There are 4 correlated columns to remove.
['tamhog', 'hhsize', 'hogar_total', 'area2']


In [9]:
hh_data.drop(to_drop, axis=1)
hh_data.columns

Index(['v2a1', 'hacdor', 'rooms', 'hacapo', 'v14a', 'refrig', 'v18q1', 'r4h1',
       'r4h2', 'r4h3', 'r4m1', 'r4m2', 'r4m3', 'r4t1', 'r4t2', 'r4t3',
       'tamhog', 'tamviv', 'hhsize', 'paredblolad', 'paredzocalo', 'paredpreb',
       'pareddes', 'paredmad', 'paredzinc', 'paredfibras', 'paredother',
       'pisomoscer', 'pisocemento', 'pisoother', 'pisonatur', 'pisonotiene',
       'pisomadera', 'techozinc', 'techoentrepiso', 'techocane', 'techootro',
       'cielorazo', 'abastaguadentro', 'abastaguafuera', 'abastaguano',
       'sanitario1', 'sanitario2', 'sanitario3', 'sanitario5', 'sanitario6',
       'energcocinar1', 'energcocinar2', 'energcocinar3', 'energcocinar4',
       'elimbasu1', 'elimbasu2', 'elimbasu3', 'elimbasu4', 'elimbasu5',
       'elimbasu6', 'hogar_nin', 'hogar_adul', 'hogar_mayor', 'hogar_total',
       'dependency', 'edjefe', 'edjefa', 'meaneduc', 'bedrooms',
       'overcrowding', 'tipovivi1', 'tipovivi2', 'tipovivi3', 'tipovivi4',
       'tipovivi5', 'computer

## Train Classifiers

In [10]:
def get_balanced_data(df, n, random_state=1):
    return df.sample(frac=1, random_state=random_state).groupby(target_column).head(n)

In [11]:
def convert_to_binary_targets(df, true_target):
    df = df.copy()
    df[target_column] = np.where(df[target_column]==true_target, 1, 0)
    return df

In [49]:
def feature_selector(selector,data, target):
    # Fit the model
    selector.fit(data, target)
    # Retain the feature names
    features = selector.get_support(indices = True)  # Returns array of indexes of nonremoved features
    k_features = [data.columns.values[i] for i in features]
    return k_features

In [52]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from data_cleaning import target_table_breakdown
from sklearn.metrics import classification_report

def run_train(clf, train_data, target_value):
    is_n = train_data.loc[train_data[target_column]<=target_value]
    is_n = convert_to_binary_targets(is_n, target_value)
    
    sel = SelectKBest(chi2, k=20)
    k_features = feature_selector(sel, is_n.drop(target_column, axis=1), is_n[target_column])
    is_n = is_n[k_features+[target_column]]
    
    sample_max = target_table_breakdown(is_n)['total'].max()
    is_n = get_balanced_data(is_n, sample_max, random_state=10)
    clf.fit(is_n.drop(target_column, axis=1), is_n[target_column])
    print(k_features)
    return clf, k_features

def run_test(clf, k_features, valid_data, target_value):    
    is_n_valid = valid_data.loc[valid_data[target_column]<=target_value]
    is_n_valid = convert_to_binary_targets(is_n_valid, target_value)
    sample_max = target_table_breakdown(is_n_valid)['total'].max()
    is_n_valid = get_balanced_data(is_n_valid, sample_max, random_state=10)
    preds = clf.predict(is_n_valid[k_features])
    print(classification_report(is_n_valid[target_column], preds))
    return list(zip(is_n_valid.index, preds))
    
def train_and_test(train_data, valid_data, target_value, clf=KNeighborsClassifier(n_neighbors=20)):
    knn, k_features = run_train(clf, train_data, target_value)
    run_test(clf, k_features, valid_data, target_value)

In [14]:
def train_all_clf(df, clfs):
    clf_4, k_features_4 = run_train(clfs.get(4), df, 4)
    clf_3, k_features_3 = run_train(clfs.get(3), df, 3)
    clf_2, k_features_2 = run_train(clfs.get(2), df, 2)
    return [(clf_2, k_features_2), (clf_3, k_features_3), (clf_4, k_features_4)]

In [15]:
from data_cleaning import compress_columns

def get_results(clf_features, df):
    [(clf_2, k_features_2), (clf_3, k_features_3), (clf_4, k_features_4)] = clf_features
    preds_2 = run_test(clf_2, k_features_2, df, 2)
    preds_3 = run_test(clf_3, k_features_3, df, 3)
    preds_4 = run_test(clf_4, k_features_4, df, 4)
    
    results_4 = pd.DataFrame(preds_4, columns=[household_id,'clf_4']).set_index(household_id)
    results_3 = pd.DataFrame(preds_3, columns=[household_id,'clf_3']).set_index(household_id)
    results_2 = pd.DataFrame(preds_2, columns=[household_id,'clf_2']).set_index(household_id)
    
    results = pd.concat([results_2, results_3, results_4], axis=1, sort=False).fillna(0).astype('int')
    
    results['clf_1'] = (~results[['clf_2','clf_3','clf_4']].any(axis=1)).astype('int')
    results = compress_columns(results, target_column, ['clf_1','clf_2','clf_3','clf_4'])+1
    results['Preds'] = results[target_column]
    all_results = pd.concat([results[['Preds']], valid_df[[target_column]]], axis=1, sort=False).fillna(0).astype('int')
    return all_results

In [16]:
import pandas as pd
valid_df = get_balanced_data(hh_data, 100)
train_df = pd.concat([hh_data.reset_index(), valid_df.reset_index()]).drop_duplicates(keep=False).set_index(household_id)

### All KNNs 20

In [17]:
clfs = {4:KNeighborsClassifier(n_neighbors=20), 
        3:KNeighborsClassifier(n_neighbors=20), 
        2:KNeighborsClassifier(n_neighbors=20)}

[(clf_2, k_features_2), (clf_3, k_features_3), (clf_4, k_features_4)] = train_all_clf(train_df, clfs)

['v2a1', 'v18q1', 'r4h1', 'r4m1', 'r4t1', 'paredblolad', 'paredmad', 'pisocemento', 'cielorazo', 'elimbasu3', 'hogar_nin', 'dependency', 'edjefe', 'edjefa', 'meaneduc', 'computer', 'roof', 'wrf', 'warning', 'possessions']
['v2a1', 'v18q1', 'r4h1', 'r4m1', 'r4t1', 'paredblolad', 'paredzinc', 'pisomoscer', 'pisocemento', 'cielorazo', 'energcocinar2', 'energcocinar4', 'hogar_nin', 'edjefe', 'edjefa', 'computer', 'walls', 'floor', 'wrf', 'warning']
['v2a1', 'r4h1', 'r4h2', 'r4h3', 'r4m1', 'r4t1', 'r4t2', 'techocane', 'abastaguafuera', 'elimbasu2', 'hogar_nin', 'hogar_adul', 'hogar_mayor', 'dependency', 'edjefe', 'edjefa', 'computer', 'qmobilephone', 'lugar3', 'roof']


In [18]:
res = get_results([(clf_2, k_features_2), (clf_3, k_features_3), (clf_4, k_features_4)], valid_df)

             precision    recall  f1-score   support

          0       0.58      0.07      0.13       100
          1       0.51      0.95      0.66       100

avg / total       0.54      0.51      0.39       200

             precision    recall  f1-score   support

          0       0.66      0.88      0.75       200
          1       0.29      0.10      0.15       100

avg / total       0.54      0.62      0.55       300

             precision    recall  f1-score   support

          0       0.92      0.51      0.66       300
          1       0.37      0.87      0.52       100

avg / total       0.78      0.60      0.63       400



In [19]:
res.head(5)

,Preds,Target
908def11b,2,2
fd750dc9b,2,1
e62755573,2,2
05eff3d5c,2,1
27f909e42,2,2


In [20]:
print(classification_report(res[target_column], res['Preds']))

             precision    recall  f1-score   support

          1       0.11      0.07      0.09       100
          2       0.51      0.95      0.66       100
          3       1.00      0.10      0.18       100
          4       0.63      0.87      0.73       100

avg / total       0.56      0.50      0.41       400



### All KNNs different n's

In [53]:
clfs = {4:KNeighborsClassifier(n_neighbors=1), 
        3:KNeighborsClassifier(n_neighbors=1), 
        2:KNeighborsClassifier(n_neighbors=3)}

t_clfs_features = train_all_clf(train_df, clfs)
res = get_results(t_clfs_features, valid_df)
print(classification_report(res[target_column], res['Preds']))

['v2a1', 'v18q1', 'r4h1', 'r4m1', 'r4t1', 'paredblolad', 'paredmad', 'pisocemento', 'cielorazo', 'elimbasu3', 'hogar_nin', 'dependency', 'edjefe', 'edjefa', 'meaneduc', 'computer', 'roof', 'wrf', 'warning', 'possessions']
['v2a1', 'v18q1', 'r4h1', 'r4m1', 'r4t1', 'paredblolad', 'paredzinc', 'pisomoscer', 'pisocemento', 'cielorazo', 'energcocinar2', 'energcocinar4', 'hogar_nin', 'edjefe', 'edjefa', 'computer', 'walls', 'floor', 'wrf', 'warning']
['v2a1', 'r4h1', 'r4h2', 'r4h3', 'r4m1', 'r4t1', 'r4t2', 'techocane', 'abastaguafuera', 'elimbasu2', 'hogar_nin', 'hogar_adul', 'hogar_mayor', 'dependency', 'edjefe', 'edjefa', 'computer', 'qmobilephone', 'lugar3', 'roof']
             precision    recall  f1-score   support

          0       0.60      0.25      0.35       100
          1       0.53      0.83      0.64       100

avg / total       0.56      0.54      0.50       200

             precision    recall  f1-score   support

          0       0.68      0.65      0.66       200
      

## Load and clean test data

In [ ]:
from data_cleaning import get_test_data
test = get_test_data()

In [107]:
hh_columns = [person_id, 'v2a1', 'hacdor', 'rooms', 'hacapo', 'v14a', 'refrig', 'v18q1', 'r4h1',
       'r4h2', 'r4h3', 'r4m1', 'r4m2', 'r4m3', 'r4t1', 'r4t2', 'r4t3',
       'tamhog', 'tamviv', 'hhsize', 'paredblolad', 'paredzocalo', 'paredpreb',
       'pareddes', 'paredmad', 'paredzinc', 'paredfibras', 'paredother',
       'pisomoscer', 'pisocemento', 'pisoother', 'pisonatur', 'pisonotiene',
       'pisomadera', 'techozinc', 'techoentrepiso', 'techocane', 'techootro',
       'cielorazo', 'abastaguadentro', 'abastaguafuera', 'abastaguano',
       'public', 'planpri', 'noelec', 'coopele', 'sanitario1', 'sanitario2',
       'sanitario3', 'sanitario5', 'sanitario6', 'energcocinar1',
       'energcocinar2', 'energcocinar3', 'energcocinar4', 'elimbasu1',
       'elimbasu2', 'elimbasu3', 'elimbasu4', 'elimbasu5', 'elimbasu6',
       'epared1', 'epared2', 'epared3', 'etecho1', 'etecho2', 'etecho3',
       'eviv1', 'eviv2', 'eviv3', 'hogar_nin', 'hogar_adul',
       'hogar_mayor', 'hogar_total', 'dependency', 'edjefe', 'edjefa',
       'meaneduc', 'bedrooms', 'overcrowding', 'tipovivi1', 'tipovivi2',
       'tipovivi3', 'tipovivi4', 'tipovivi5', 'computer', 'television',
       'qmobilephone', 'lugar1', 'lugar2', 'lugar3', 'lugar4', 'lugar5',
       'lugar6', 'area1', 'area2', 'v2a1-missing', 'rez_esc-missing']

In [108]:
test = clean_data(test)
test = test.reset_index()[hh_columns] #.groupby(household_id).mean()
test = compress_column_data(test)
test = add_custom_features(test).set_index(person_id)

## Run predictions

In [114]:
def run_preds(clf, k_features, df):    
    print(k_features)
    preds = clf.predict(df[k_features])
    return list(zip(df.index, preds))

In [117]:
def get_predictions(clf_features, df):
    [(clf_2, k_features_2), (clf_3, k_features_3), (clf_4, k_features_4)] = clf_features
    preds_2 = run_preds(clf_2, k_features_2, df)
    preds_3 = run_preds(clf_3, k_features_3, df)
    preds_4 = run_preds(clf_4, k_features_4, df)
    
    results_4 = pd.DataFrame(preds_4, columns=[person_id,'clf_4']).set_index(person_id)
    results_3 = pd.DataFrame(preds_3, columns=[person_id,'clf_3']).set_index(person_id)
    results_2 = pd.DataFrame(preds_2, columns=[person_id,'clf_2']).set_index(person_id)
    
    results = pd.concat([results_2, results_3, results_4], axis=1, sort=False).fillna(0).astype('int')
    
    results['clf_1'] = (~results[['clf_2','clf_3','clf_4']].any(axis=1)).astype('int')
    return results


In [118]:
results = get_predictions(t_clfs_features, test)

['v2a1', 'r4h1', 'r4h2', 'r4h3', 'r4m1', 'r4t1', 'r4t2', 'techocane', 'abastaguafuera', 'elimbasu2', 'hogar_nin', 'hogar_adul', 'hogar_mayor', 'dependency', 'edjefe', 'edjefa', 'computer', 'qmobilephone', 'lugar3', 'roof']
['v2a1', 'v18q1', 'r4h1', 'r4m1', 'r4t1', 'paredblolad', 'paredzinc', 'pisomoscer', 'pisocemento', 'cielorazo', 'energcocinar2', 'energcocinar4', 'hogar_nin', 'edjefe', 'edjefa', 'computer', 'walls', 'floor', 'wrf', 'warning']
['v2a1', 'v18q1', 'r4h1', 'r4m1', 'r4t1', 'paredblolad', 'paredmad', 'pisocemento', 'cielorazo', 'elimbasu3', 'hogar_nin', 'dependency', 'edjefe', 'edjefa', 'meaneduc', 'computer', 'roof', 'wrf', 'warning', 'possessions']


In [120]:
targets = compress_columns(results, target_column, ['clf_1','clf_2','clf_3','clf_4'])+1

In [122]:
targets[target_column].value_counts()

2    20011
4     1483
3     1403
1      959
Name: Target, dtype: int64

In [123]:
targets.to_csv('knns.csv')